In [3]:
import pandas as pd 
import warnings
from sklearn import datasets
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
warnings.simplefilter('ignore')


In [5]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
X = iris.data 
y = iris.target 
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)


In [12]:
model = SVC(C=100, kernel='rbf', gamma=10)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
y_predict


array([1, 2, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0,
       0, 1, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 0, 1, 0,
       1, 2, 0, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0,
       0, 2])

In [14]:
accuracy = accuracy_score(y_test, y_predict)
accuracy

0.9

GridSearchCV

In [16]:
param_grid_gridsearch = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.01, 0.1, 1, 'auto']
}
model_gridsearch = SVC()
grid_search = GridSearchCV(model_gridsearch,
                           param_grid=param_grid_gridsearch,
                           scoring='accuracy',
                           cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.01, 0.1, 1, 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [18]:
best_params_grid = grid_search.best_params_
best_params_grid

{'C': 10, 'gamma': 0.01, 'kernel': 'linear'}

In [19]:
best_model_grid = grid_search.best_estimator_
y_pred_grid = best_model_grid.predict(X_test)

accuracy_grid = accuracy_score(y_test, y_pred_grid)
accuracy_grid

0.9777777777777777